# Assessment

## Questions

1. How many facilities do we have?
2. How many premium facilities do we have?
3. What’s the facilities  distribution by size?
4. How many valid phones do we have?
5. What’s the state with the most premium facilities as of today?
6. How many facilities have churned since June 2022?
7. What’s the top 3 states with the highest churn percentage?
8. What’s the churn probability for facilities in its 5th month (at national level)?
9. What’s the top 3 states with the most invalid phone number proportion?
10. What’s the average facilities lifespan?
11. What’s the top 3 states with the biggest facilities?
12. Do we have duplicated phone numbers?
13. What’s the top 3 valid duplicated numbers?
14. What’s the top 3 states with the most valid duplicated numbers?
15. What is the relation between valid/invalid phones with churn propensity?

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime

# Data Acquisition

In [12]:
df = pd.read_csv("CS_Ops_Assessment_dataset_JuanReyes.csv")

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631 entries, 0 to 1630
Data columns (total 10 columns):
 #   Column                                                                                            Non-Null Count  Dtype  
---  ------                                                                                            --------------  -----  
 0   facility_id                                                                                       1631 non-null   int64  
 1   facility_category                                                                                 1631 non-null   object 
 2   facility_size                                                                                     1631 non-null   object 
 3   state                                                                                             1631 non-null   object 
 4   city                                                                                              1631 non-null   object 
 5  

In [14]:
df.head()

,facility_id,facility_category,facility_size,state,city,phone,is_premium,premium_since,is_churn,churn_since
0,80,consultorios de medicina general del sector pr...,0 a 5 personas,chiapas,san cristobal de las casas ...,0.000000e+00,1,9/8/2020,0,na
1,160,consultorios del sector privado de audiologia ...,0 a 5 personas,guanajuato,leon ...,1.772161e+09,0,na,na,na
2,240,consultorios de medicina general del sector pr...,0 a 5 personas,morelos,tepoztlan ...,0.000000e+00,1,5/27/2020,1,9/30/2021
3,320,laboratorios medicos y de diagnostico del sect...,0 a 5 personas,queretaro,queretaro ...,0.000000e+00,0,na,na,na
4,400,consultorios dentales del sector privado ...,0 a 5 personas,veracruz de ignacio de la llave,xalapa ...,2.261960e+09,1,12/11/2021,0,na


In [15]:
df.tail()

,facility_id,facility_category,facility_size,state,city,phone,is_premium,premium_since,is_churn,churn_since
1626,130160,consultorios dentales del sector privado ...,0 a 5 personas,zacatecas,ojocaliente ...,0.000000e+00,0,na,na,na
1627,130240,consultorios de medicina general del sector pr...,0 a 5 personas,zacatecas,tepetongo ...,1.919111e+07,1,5/22/2022,1,6/12/2022
1628,130320,consultorios dentales del sector privado ...,0 a 5 personas,zacatecas,guadalupe ...,1.921662e+09,1,11/22/2020,0,na
1629,130400,laboratorios medicos y de diagnostico del sect...,0 a 5 personas,zacatecas,fresnillo ...,1.939337e+07,1,3/7/2022,0,na
1630,130480,consultorios de medicina general del sector pr...,0 a 5 personas,zacatecas,calera ...,1.769620e+05,0,na,na,na


In [16]:
#Let's get rid of the trailing white spaces of the columns
df.columns = df.columns.str.strip()
#Let's get rid of the trailing white spaces of the all the rows
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [41]:
pd.set_option('display.max_rows', None)

In [17]:
#Visualize the unique values per column of interest
print(f"""
Unique Facility Categories:
{df['facility_category'].unique()}

Unique Facility Sizes:
{df['facility_size'].unique()}

Unique States:
{df['state'].unique()}

Unique Cities:
{df['city'].unique()}

Unique Is_premium:
{df["is_premium"].unique()}
""")



Unique Facility Categories:
['consultorios de medicina general del sector privado'
 'consultorios del sector privado de audiologia y de terapia ocupacional, fisica y del lenguaje'
 'laboratorios medicos y de diagnostico del sector privado'
 'consultorios dentales del sector privado'
 'consultorios de medicina especializada del sector privado'
 'consultorios de optometria'
 'clinicas de consultorios medicos del sector privado'
 'consultorios de nutriologos y dietistas del sector privado'
 'hospitales generales del sector privado'
 'consultorios de psicologia del sector privado'
 'consultorios de quiropractica del sector privado'
 'otros consultorios del sector privado para el cuidado de la salud'
 'hospitales del sector privado de otras especialidades medicas']

Unique Facility Sizes:
['0 a 5 personas' '6 a 10 personas' '11 a 30 personas' '31 a 50 personas'
 '101 a 250 personas' '251 y mas personas' '51 a 100 personas']

Unique States:
['chiapas' 'guanajuato' 'morelos' 'queretaro'
 've

# Initial Data Analysis (IDA)

In [18]:
#Replace standalone word 'na' not words that contain 'nan'
df = df.replace(r'\bna\b', np.nan, regex=True)

In [19]:
#Replace NaN for NaT & ensuring dates are in datetime64
def parse_date_with_multiple_formats(date_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        return pd.to_datetime(date_str, format="%m/%d/%Y")
    except ValueError:
        print(f"Failed to parse '{date_str}'")
        pass

    return pd.NaT

df['churn_since'] = df['churn_since'].apply(parse_date_with_multiple_formats)
df['premium_since'] = df['premium_since'].apply(parse_date_with_multiple_formats)

In [20]:
df['is_churn'] = df['is_churn'].astype('Int64') #We do this to handle NaNs & to ensure nums are int

In [21]:
df['is_premium'] = df['is_premium'].astype(int)  # Ensure it is an int

In [22]:
df['phone'] = df['phone'].astype(int) #If we leave it as float there are mistakes
df['phone'] = df['phone'].astype(str) #We need this to determine valid numbers

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631 entries, 0 to 1630
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   facility_id        1631 non-null   int64         
 1   facility_category  1631 non-null   object        
 2   facility_size      1631 non-null   object        
 3   state              1631 non-null   object        
 4   city               1631 non-null   object        
 5   phone              1631 non-null   object        
 6   is_premium         1631 non-null   int64         
 7   premium_since      799 non-null    datetime64[ns]
 8   is_churn           799 non-null    Int64         
 9   churn_since        380 non-null    datetime64[ns]
dtypes: Int64(1), datetime64[ns](2), int64(2), object(5)
memory usage: 129.1+ KB


# EDA

In [78]:
df.describe()

,facility_id,is_premium,premium_since,is_churn,churn_since,tenure_days,tenure_months
count,1631.000000,1631.000000,799,799.0,380,1631.000000,1631.000000
mean,65280.000000,0.489884,2021-06-13 22:29:53.241551872,0.475594,2022-04-02 08:12:37.894736896,321.190067,10.322502
min,80.000000,0.000000,2020-01-02 00:00:00,0.0,2020-01-12 00:00:00,-1.000000,0.000000
25%,32680.000000,0.000000,2020-10-14 00:00:00,0.0,2021-11-29 18:00:00,-1.000000,0.000000
50%,65280.000000,0.000000,2021-06-08 00:00:00,0.0,2022-06-05 12:00:00,-1.000000,0.000000
75%,97880.000000,1.000000,2022-02-12 12:00:00,1.0,2022-10-01 00:00:00,601.500000,19.000000
max,130480.000000,1.000000,2022-11-30 00:00:00,1.0,2022-12-30 00:00:00,1516.000000,49.000000
std,37677.876798,0.500051,NaN,0.499717,NaN,459.033563,14.892135


# 1. How many facilities do we have?

In [24]:
#Assuming facility_id is a primary key then the number of facilities is simply the total number of unique entries
total_facilities = df["facility_id"].nunique()

print(total_facilities)


1631


# 2. How many premium facilities do we have?

In [25]:
#Two approaches
total_premium_facilities = df['is_premium'].value_counts().get(1, 0)
total_premium_facilities_query = df.query('is_premium == 1').shape[0]


In [26]:
print(total_premium_facilities)
print(total_premium_facilities_query)

799
799


# 3. What’s the facilities distribution by size?

In [27]:
# Mimicking a groupby operation
facility_size_distribution = df['facility_size'].value_counts()
print(facility_size_distribution)


facility_size
0 a 5 personas        1519
6 a 10 personas         74
11 a 30 personas        25
31 a 50 personas         8
101 a 250 personas       3
251 y mas personas       1
51 a 100 personas        1
Name: count, dtype: int64


# 4. How many valid phones do we have?


In [28]:
#All valid numbers in MX need to have 10 digits: https://telmex.com/10digitos

valid_phones_df = df.loc[df['phone'].str.len() == 10]
number_of_valid_phones = valid_phones_df.shape[0]


print(f"Number of valid phone numbers with exactly 10 digits: {number_of_valid_phones}")


Number of valid phone numbers with exactly 10 digits: 440


# 5. What’s the state with the most premium facilities as of today?

In [29]:
premium_counts_by_state = df[df['is_premium'] == 1]['state'].value_counts()
print(f"Counts of premium facilities by state:{premium_counts_by_state}")

most_premium_state = premium_counts_by_state.idxmax()
print(f"\nThe state with the most premium facilities is: {most_premium_state} with {premium_counts_by_state.max()} premium facilities.")


Counts of premium facilities by state:state
mexico                             98
ciudad de mexico                   71
jalisco                            51
veracruz de ignacio de la llave    49
puebla                             45
michoacan de ocampo                41
nuevo leon                         38
guanajuato                         35
oaxaca                             33
chihuahua                          30
baja california                    28
sinaloa                            28
chiapas                            26
tamaulipas                         21
morelos                            19
queretaro                          18
coahuila de zaragoza               17
guerrero                           17
sonora                             16
yucatan                            16
hidalgo                            15
san luis potosi                    13
zacatecas                          12
nayarit                            10
tlaxcala                            9
aguasc

# 6. How many facilities have churned since June 2022?

In [30]:
churn_since_june_df = df[(df['is_churn'] == 1) & (df['churn_since'] > pd.to_datetime('2022-06-01'))]


In [31]:
churn_since_june_df.head()

,facility_id,facility_category,facility_size,state,city,phone,is_premium,premium_since,is_churn,churn_since
11,960,consultorios dentales del sector privado,0 a 5 personas,queretaro,queretaro,0,1,2022-09-19,1,2022-11-16
16,1360,clinicas de consultorios medicos del sector pr...,0 a 5 personas,aguascalientes,aguascalientes,1191109600,1,2021-04-07,1,2022-11-05
26,2160,clinicas de consultorios medicos del sector pr...,0 a 5 personas,nuevo leon,monterrey,6163691111,1,2021-08-16,1,2022-08-18
32,2640,consultorios dentales del sector privado,0 a 5 personas,sinaloa,el fuerte,0,1,2021-10-17,1,2022-06-18
34,2800,consultorios de quiropractica del sector privado,0 a 5 personas,tlaxcala,tlaxcala,0,1,2022-10-28,1,2022-12-25


In [32]:
churn_since_june_df["facility_id"].nunique()

194

# 7. What’s the top 3 states with the highest churn percentage?

In [33]:

churn_percentage_by_state = df.groupby('state')['is_churn'].mean() * 100
sorted_states = churn_percentage_by_state.sort_values(ascending=False)
top_3_states = sorted_states.head(3)

print(sorted_states, "\n\n", top_3_states)

state
campeche                               100.0
quintana roo                       83.333333
chiapas                            61.538462
san luis potosi                    61.538462
mexico                             59.183673
chihuahua                          56.666667
sonora                                 56.25
queretaro                          55.555556
baja california                    53.571429
ciudad de mexico                   53.521127
morelos                            52.631579
zacatecas                               50.0
baja california sur                     50.0
nuevo leon                         47.368421
oaxaca                             45.454545
aguascalientes                     44.444444
tlaxcala                           44.444444
durango                            42.857143
michoacan de ocampo                41.463415
coahuila de zaragoza               41.176471
veracruz de ignacio de la llave    40.816327
hidalgo                                 40.0
pueb

In [34]:
campeche = df.query('state == "campeche"')
campeche

,facility_id,facility_category,facility_size,state,city,phone,is_premium,premium_since,is_churn,churn_since
77,6240,clinicas de consultorios medicos del sector pr...,11 a 30 personas,campeche,escarcega,9626213122,0,NaT,<NA>,NaT
80,6480,laboratorios medicos y de diagnostico del sect...,0 a 5 personas,campeche,calkini,0,1,2021-08-10,1,2021-09-18
81,6560,consultorios del sector privado de audiologia ...,0 a 5 personas,campeche,calkini,0,0,NaT,<NA>,NaT
90,7280,consultorios dentales del sector privado,0 a 5 personas,campeche,carmen,9361377696,0,NaT,<NA>,NaT
95,7680,laboratorios medicos y de diagnostico del sect...,0 a 5 personas,campeche,campeche,9611112122,0,NaT,<NA>,NaT
828,66320,consultorios de medicina especializada del sec...,0 a 5 personas,campeche,carmen,93636696,0,NaT,<NA>,NaT
1154,92400,consultorios dentales del sector privado,0 a 5 personas,campeche,calkini,0,0,NaT,<NA>,NaT
1155,92480,consultorios de medicina especializada del sec...,0 a 5 personas,campeche,campeche,6163669161,0,NaT,<NA>,NaT


# 8. What’s the churn probability for facilities in its 5th month (at national level)?


In [37]:
df.head()

,facility_id,facility_category,facility_size,state,city,phone,is_premium,premium_since,is_churn,churn_since
0,80,consultorios de medicina general del sector pr...,0 a 5 personas,chiapas,san cristobal de las casas,0,1,2020-09-08,0,NaT
1,160,consultorios del sector privado de audiologia ...,0 a 5 personas,guanajuato,leon,1772161292,0,NaT,<NA>,NaT
2,240,consultorios de medicina general del sector pr...,0 a 5 personas,morelos,tepoztlan,0,1,2020-05-27,1,2021-09-30
3,320,laboratorios medicos y de diagnostico del sect...,0 a 5 personas,queretaro,queretaro,0,0,NaT,<NA>,NaT
4,400,consultorios dentales del sector privado,0 a 5 personas,veracruz de ignacio de la llave,xalapa,2261960106,1,2021-12-11,0,NaT


In [77]:
reference_date = datetime.now()

df['tenure_days'] = np.where(
    pd.notnull(df['churn_since']),
    (df['churn_since'] - df['premium_since']).dt.days,
    (reference_date - df['premium_since']).dt.days
)

# Fill NaN values that result from NaT entries with the placeholder -1
df['tenure_days'] = df['tenure_days'].fillna(-1)

df['tenure_months'] = (df['tenure_days'] / 30.44).astype(int)


#Starts on 0 so 5th month is 4. Also, this includes facilities that churned exactly on the 5th month mark
facilities_in_5th_month = df[(df['tenure_months'] == 4)]
facilities_in_5th_month.describe()

,facility_id,is_premium,premium_since,is_churn,churn_since,tenure_days,tenure_months
count,26.000000,26.0,26,26.0,26,26.000000,26.0
mean,70581.538462,1.0,2021-11-22 04:36:55.384615424,1.0,2022-04-07 21:13:50.769230848,136.692308,4.0
min,1760.000000,1.0,2020-06-18 00:00:00,1.0,2020-11-16 00:00:00,122.000000,4.0
25%,45940.000000,1.0,2021-09-06 12:00:00,1.0,2022-01-14 18:00:00,128.250000,4.0
50%,79480.000000,1.0,2021-12-07 12:00:00,1.0,2022-04-28 00:00:00,136.500000,4.0
75%,96780.000000,1.0,2022-06-09 18:00:00,1.0,2022-10-24 00:00:00,145.500000,4.0
max,126800.000000,1.0,2022-08-11 00:00:00,1.0,2022-12-30 00:00:00,151.000000,4.0
std,34355.003559,0.0,NaN,0.0,NaN,9.540521,0.0


In [76]:
churned_in_5th_month = facilities_in_5th_month[facilities_in_5th_month['is_churn'] == 1].shape[0]
total_premium_facilities = df[pd.notnull(df['premium_since'])].shape[0]

churn_probability_5th_month = churned_in_5th_month / total_premium_facilities

print(f"The churn probability for a facility in its 5th month is: {churn_probability_5th_month:.2%}")


The churn probability for a facility in its 5th month is: 3.25%


# 9. What’s the top 3 states with the most invalid phone number proportion?

In [97]:
df['invalid_phone'] = 0

df.loc[df['phone'].str.len() != 10, 'invalid_phone'] = 1

state_phone_stats = df.groupby('state')['invalid_phone'].agg(['sum', 'count'])

state_phone_stats["invalid_phone_proportion"] = state_phone_stats["sum"] / state_phone_stats["count"]

top_invalid_phone_states = state_phone_stats.sort_values(by='invalid_phone_proportion', ascending=False)

top_invalid_phone_states.head(3)

,sum,count,invalid_phone_proportion
state,,,
tabasco,21,23,0.913043
colima,9,10,0.900000
guerrero,26,29,0.896552


# 10. What’s the average facilities lifespan?


In [101]:
df.describe()

,facility_id,is_premium,premium_since,is_churn,churn_since,tenure_days,tenure_months,invalid_phone
count,1631.000000,1631.000000,799,799.0,380,1631.000000,1631.000000,1631.000000
mean,65280.000000,0.489884,2021-06-13 22:29:53.241551872,0.475594,2022-04-02 08:12:37.894736896,321.190067,10.322502,0.730227
min,80.000000,0.000000,2020-01-02 00:00:00,0.0,2020-01-12 00:00:00,-1.000000,0.000000,0.000000
25%,32680.000000,0.000000,2020-10-14 00:00:00,0.0,2021-11-29 18:00:00,-1.000000,0.000000,0.000000
50%,65280.000000,0.000000,2021-06-08 00:00:00,0.0,2022-06-05 12:00:00,-1.000000,0.000000,1.000000
75%,97880.000000,1.000000,2022-02-12 12:00:00,1.0,2022-10-01 00:00:00,601.500000,19.000000,1.000000
max,130480.000000,1.000000,2022-11-30 00:00:00,1.0,2022-12-30 00:00:00,1516.000000,49.000000,1.000000
std,37677.876798,0.500051,NaN,0.499717,NaN,459.033563,14.892135,0.443978


In [105]:
avg_lifespan = df["tenure_days"].mean()

print(f"The average facilities lifespan is {avg_lifespan:.0f} days.")

The average facilities lifespan is 321 days.
